## Loan Default Prediction

> ****Financial institutions, including banks, credit unions, and government lenders, provide loan services to individuals and businesses.
A major concern for these institutions is loan defaults, where borrowers fail to repay their loans as agreed.
To manage this risk, many institutions utilize machine learning algorithms to predict which borrowers are most likely to default.****

> The Dataset:
This specific dataset originates from Coursera's Loan Default Prediction Challenge.
It offers an opportunity to practice and test machine learning skills on a real-world problem.
The dataset comprises a significant amount of data, with 255,347 rows and 18 columns.
Presumably, these columns contain information relevant to borrowers and their loan characteristics.

> The Goal:
The primary objective is to utilize the data to build a model that accurately predicts loan defaults.
This would enable financial institutions to:
Identify high-risk borrowers at an early stage.
Implement targeted interventions, such as loan restructuring or financial counseling, to prevent defaults.
Make more informed lending decisions, potentially reducing overall risk exposure.

In [ ]:
# load the data 
import numpy as np
import pandas as pd
loan=pd.read_csv("Loan_default.csv")

In [ ]:
loan.head()

In [ ]:
#info
loan.info()

In [ ]:
#label encoding
from sklearn.preprocessing import LabelEncoder
# Assuming df is your DataFrame
categorical_columns = ['Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner']

# Initializing LabelEncoder
label_encoder = LabelEncoder()

# Encoding categorical columns
for col in categorical_columns:
    loan[col] = label_encoder.fit_transform(loan[col])

In [ ]:
#EDA
import seaborn as sns
import matplotlib.pyplot as plt
columns = loan.columns
fig, axs = plt.subplots(5, 2, figsize=(10, 10))
sns.boxplot(loan.iloc[:,1],ax=axs[0, 0],orient='h').set_title(columns[1])
sns.boxplot(loan.iloc[:,2],ax=axs[0, 1],orient='h').set_title(columns[2])
sns.boxplot(loan.iloc[:,3],ax=axs[1, 0],orient='h').set_title(columns[3])
sns.boxplot(loan.iloc[:,4],ax=axs[1, 1],orient='h').set_title(columns[4])
sns.boxplot(loan.iloc[:,5],ax=axs[2, 0],orient='h').set_title(columns[5])
sns.boxplot(loan.iloc[:,6],ax=axs[2, 1],orient='h').set_title(columns[6])
sns.boxplot(loan.iloc[:,7],ax=axs[3, 0],orient='h').set_title(columns[7])
sns.boxplot(loan.iloc[:,8],ax=axs[3, 1],orient='h').set_title(columns[8])
sns.boxplot(loan.iloc[:,9],ax=axs[4, 0],orient='h').set_title(columns[9])
fig.tight_layout()

In [ ]:
# Check if our observations are correct
np.round(loan.pivot_table(index='Default',values=['InterestRate','LoanAmount','NumCreditLines'],aggfunc=('mean','std')),2)

In [ ]:
# Check if our observations are correct
np.round(loan.pivot_table(index='Default',values=['Age','Income','MonthsEmployed'],aggfunc=('mean','std')),2)

In [ ]:
loan.isnull()

In [ ]:
# Split Dataset into train and test dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(loan.drop(columns=['LoanID','Default']),loan['Default'], test_size=0.25,random_state=42)

print('x_train:',x_train.shape)
print('y_train:',y_train.shape)
print('x_test:',x_test.shape)
print('y_test:',y_test.shape)

## ML models - Ensemble methods

> #### Ensemble methods
     Bagging
     Boosting
     Stacking
     Voting

In [ ]:
from sklearn.ensemble import  BaggingClassifier, GradientBoostingClassifier,RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

### Bagging Method

In [ ]:
# Create a base classifier (e.g., DecisionTreeClassifier):
base_clf = DecisionTreeClassifier()

# Create the bagging classifier:
bagging_clf = BaggingClassifier(base_estimator=base_clf, n_estimators=100, random_state=42)

# Train and evaluate (similar to Voting):
bagging_clf.fit(x_train, y_train)
y_pred = bagging_clf.predict(x_test)
bagging_accuracy = accuracy_score(y_test, y_pred)
print("Bagging accuracy:", bagging_accuracy)


### Boosting method

In [ ]:
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb_clf.fit(x_train, y_train)
y_pred = gb_clf.predict(x_test)
boosting_accuracy = accuracy_score(y_test, y_pred)
print("Boosting accuracy:", boosting_accuracy)


### stacking Method

In [ ]:
# Define the base models
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42))
]

# Define the meta-model
meta_model = LogisticRegression()

# Create the stacking classifier
stacking_classifier = StackingClassifier(estimators=base_models, final_estimator=meta_model)

# Train the stacking classifier on the training data
stacking_classifier.fit(x_train, y_train)

# Make predictions on the test set
stacking_predictions = stacking_classifier.predict(x_test)

# Evaluate the performance
stack_accuracy = accuracy_score(y_test, stacking_predictions)
print(f'Accuracy of Stacking Classifier: {stack_accuracy}')

## Tree based Learning Methods

### Random Forest:

In [ ]:


rf_clf = RandomForestClassifier(n_estimators=100,  random_state=42)

# Train and evaluate:
rf_clf.fit(x_train, y_train)
y_pred = rf_clf.predict(x_test)
random_accuracy = accuracy_score(y_test, y_pred)
print("Random Forest accuracy:", random_accuracy)




###  Decision Tree:

In [ ]:

dt_clf = DecisionTreeClassifier(random_state=42)

# Train and evaluate:
dt_clf.fit(x_train, y_train)
y_pred = dt_clf.predict(x_test)
dt_accuracy = accuracy_score(y_test, y_pred)
print("Decision Tree accuracy:", dt_accuracy)




In [ ]:
import matplotlib.pyplot as plt

# Accuracy scores for different models
accuracies = {
    "Decision Tree": dt_accuracy,
    "Random Forest": random_accuracy,
    "Stacking": stack_accuracy,
    "Boosting": boosting_accuracy,
    "Bagging": bagging_accuracy
}


accuracies = dict(sorted(accuracies.items(), key=lambda item: item[1], reverse=True))

# Create the bar chart
plt.figure(figsize=(16, 9))  # Adjust figure size as needed

colors = ['lightblue' if model != 'Boosting' else 'orange' for model in accuracies.keys()]
plt.bar(accuracies.keys(), accuracies.values(), color=colors)
# Customize chart elements
plt.xlabel("Model Type", fontsize=14)
plt.ylabel("Accuracy", fontsize=14)
plt.title("Accuracy Comparison of Different Models", fontsize=18)
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.ylim(0, 1.1)  # Set y-axis limits for precision

plt.grid(axis='y', linestyle='--', alpha=0.8)  # Add grid lines

# Annotate each bar with 8 decimal places
for i, v in enumerate(accuracies.values()):
    plt.text(i, v + 0.005, f"{v:.4f}")  # Format with 8 decimals

plt.tight_layout()  # Adjust layout for better spacing
plt.show()


## voting classifier

In [ ]:

from sklearn.ensemble import VotingClassifier

# Create a list of tuples with (model_name, model_instance)
models = [ ( "Decision Tree", dt_clf), ("Random Forest",rf_clf), ( "Stacking", stacking_classifier), 
          ("Boosting", gb_clf),("Bagging", bagging_clf)]

# Create a Voting Classifier
voting_classifier = VotingClassifier(estimators=models, voting='hard')  # 'hard' for majority voting


# Fit the Voting Classifier on the training data
voting_classifier.fit(x_train, y_train)

# Predictions
predictions = voting_classifier.predict(x_test)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, predictions)
print(f'Ensemble Model Accuracy: {accuracy}')




In [ ]:
# Create the bar chart
# Calculate and print accuracy
accuracy_ensemble = accuracy_score(y_test, predictions)
print(f'Ensemble Model Accuracy: {accuracy}')

In [ ]:
import matplotlib.pyplot as plt

# Accuracy scores for different models
accuracies = {
    "Decision Tree": dt_accuracy,
    "Random Forest": random_accuracy,
    "Stacking": stack_accuracy,
    "Boosting": boosting_accuracy,
    "Bagging": bagging_accuracy,
    "Voting":accuracy_ensemble  # Add accuracy for the ensemble model
}

# Sort the accuracies dictionary by values in descending order
accuracies = dict(sorted(accuracies.items(), key=lambda item: item[1], reverse=True))

# Create the bar chart
plt.figure(figsize=(16, 9))  # Adjust figure size as needed

# Define colors for each bar
colors = ['lightblue' if model != 'Voting' else 'red' for model in accuracies.keys()]

# Plot the bars
plt.bar(accuracies.keys(), accuracies.values(), color=colors)

# Customize chart elements
plt.xlabel("Model Type", fontsize=14)
plt.ylabel("Accuracy", fontsize=14)
plt.title("Accuracy Comparison of Different Models", fontsize=18)
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.ylim(0, 1.1)  # Set y-axis limits for precision

plt.grid(axis='y', linestyle='--', alpha=0.8)  # Add grid lines

# Annotate each bar with 8 decimal places
for i, v in enumerate(accuracies.values()):
    plt.text(i, v + 0.005, f"{v:.4f}")  # Format with 8 decimals

plt.tight_layout()  # Adjust layout for better spacing
plt.show()
